In [46]:
# Lib
import pandas as pd
import numpy as np



In [47]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [48]:
# Dataset

df = pd.read_csv('./data/tripadvisor_airlinereviews_binary.csv', encoding='UTF-8')
df.head()

,Unnamed: 0,_id,title,text,rating,reviewer_handle,language,airline_name,flight_date,flight_connection,flight_type,booking_class,sentiment
0,0,BAFHFJ4n,NaN,Ich bin sehr positiv angetan mit diese Gesells...,5.0,NaN,NaN,Emirates,NaN,NaN,NaN,NaN,Positive
1,2,gUyMgdC3,NaN,Flug wurde von Sbg ursprünglich gebucht. Wurde...,2.0,NaN,NaN,Emirates,NaN,NaN,NaN,NaN,Negative
2,3,MHjJQCeQ,NaN,Wir sind um 15 Uhr 30 in den Flieger gestiegen...,2.0,NaN,NaN,Emirates,NaN,NaN,NaN,NaN,Negative
3,5,HF1dz34d,NaN,Ich und mein Freund flogen Mitte April nach Du...,1.0,NaN,NaN,Emirates,NaN,NaN,NaN,NaN,Negative
4,6,VLoFweME,NaN,Rückreise aus Neuseeland war folgenderweise ge...,1.0,NaN,NaN,Emirates,NaN,NaN,NaN,NaN,Negative


In [49]:
def load_imdb_data(df):
    texts = df['text'].tolist()
    labels = [1 if sentiment == "Positive" else 0 for sentiment in df['sentiment'].tolist()]
    return texts, labels

In [50]:
texts, labels = load_imdb_data(df)

In [51]:
# Create Custom Dataset Class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [52]:
# Classifier
class BERTClassifier(nn.Module):

    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [53]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model.forward(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [54]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model.forward(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [55]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
            outputs = model.forward(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
    return "positive" if preds.item() == 1 else "negative"

In [62]:
# Params
bert_model_name = 'dbmdz/bert-base-german-cased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 1
learning_rate = 2e-5

In [57]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [58]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

In [60]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

c:\Users\lucas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [63]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/1
Validation Accuracy: 0.9530
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       195
           1       0.96      0.93      0.95       167

    accuracy                           0.95       362
   macro avg       0.95      0.95      0.95       362
weighted avg       0.95      0.95      0.95       362



In [64]:
# Save model
torch.save(model.state_dict(), "bert_binary_sentiment_classifier.pth")

In [66]:
# Test sentiment prediction
test_review = """Sehr netter Service an Bord. Besonders Kinder bekommen an Bord volle Aufmerksamkeit.
Sehr leckeres und von der Menge ausreichendes Essen.
Alle Getränke sind inklusive und das so viel wie man möchte.
Gutes Unterhaltungsprogramm.
Sehr hübsche Stewardessen."""
sentiment = predict_sentiment(test_review, model, tokenizer, device)
print(test_review)
print(f"Predicted sentiment: {sentiment}")

Sehr netter Service an Bord. Besonders Kinder bekommen an Bord volle Aufmerksamkeit.
Sehr leckeres und von der Menge ausreichendes Essen.
Alle Getränke sind inklusive und das so viel wie man möchte.
Gutes Unterhaltungsprogramm.
Sehr hübsche Stewardessen.
Predicted sentiment: positive


In [67]:
neg_review = """Flug wurde von Sbg ursprünglich gebucht. Wurde seitens Emirates storniert und die dadurch entstanden Mehrkosten nicht erstattet. 
Kosten mind. 300 Euro mehr und man hatte keine Chance auf Erstattung. Wurde mit einer Lounge Einladung (Wert zu dem Zeitpunkt 25, - pro Person), jedoch nur für Wien. 
Umbuchung aufgrund Hotel bzw. Inlandsflügen in Thailand nicht möglich. In Zukunft lieber nicht mehr. Service schlecht zu diesem Thema."""
sentiment = predict_sentiment(neg_review, model, tokenizer, device)
print(neg_review)
print(f"Predicted sentiment: {sentiment}")


Flug wurde von Sbg ursprünglich gebucht. Wurde seitens Emirates storniert und die dadurch entstanden Mehrkosten nicht erstattet. 
Kosten mind. 300 Euro mehr und man hatte keine Chance auf Erstattung. Wurde mit einer Lounge Einladung (Wert zu dem Zeitpunkt 25, - pro Person), jedoch nur für Wien. 
Umbuchung aufgrund Hotel bzw. Inlandsflügen in Thailand nicht möglich. In Zukunft lieber nicht mehr. Service schlecht zu diesem Thema.
Predicted sentiment: negative
